In [1]:
import sys
if not sys.warnoptions:
    import warnings
    warnings.simplefilter("ignore")

import os
import numpy as np
import pandas as pd
import tensorflow as tf;
import tensorflow_hub as hub

import spacy
from spacy.lang.en import English
from spacy import displacy

# Load the spaCy model
# nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
nlp = spacy.load("en_core_web_sm")

# Set a larger max_length limit
# nlp.max_length = 35000000
# nlp.add_pipe('sentencizer')

import logging
logging.getLogger('tensorflow').disabled = True 

In [2]:
df = pd.read_excel('/kaggle/input/marvel-dataset/Marvel Datastore.xlsx')
df.head()

,Description
0,"Thor Odinson is the Asgardian God of Thunder, ..."
1,"Upon being welcomed back to Asgard as a hero, ..."
2,Thor returned to Asgard having defeated his br...
3,Loki Laufeyson was the biological son of Laufe...
4,"Transported by the wormhole to Sanctuary, Loki..."


In [3]:
!pip install mteb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.1/102.1 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 518.9/518.9 kB 13.1 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 8.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=d015c36cfd18b6288a360e9e9ed34d14f9a78f09d85c405a6da85194252a067f
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers
  Attempting uninstall: datasets
    Found existing installation: datasets 2.1.0
    Uninstalling datasets-2.1.0:
      Successfully uninstalled datasets-2.1.0


In [4]:
pip install mteb[beir]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 2.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 46.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 219.2/219.2 kB 18.3 MB/s eta 0:00:00
  Created wheel for beir: filename=beir-2.0.0-py3-none-any.whl size=63570 sha256=b9e641900652a88336ba4cc30ed3f53187c86ec86af23f6242382be26cde6565
  Stored in directory: /root/.cache/pip/wheels/1c/14/96/c606ede3c10e9300ef771a6183af09d389459195ff5f854862
  Created wheel for pytrec_eval: filename=pytrec_eval-0.5-cp310-cp310-linux_x86_64.whl size=308136 sha256=1d54c9b1e6c07d483f55310eacb136fe61896eb67e3f1ec9d6fc3be3638c8fa9
  Stored in directory: /root/.cache/pip/wheels/51/3a/cd/dcc1ddfc763987d5cb237165d8ac249aa98a23ab90f67317a8
Successfully built beir pytrec_eval
Note: you may need to restart the kernel to use updated packages.


In [5]:

from mteb.evaluation.evaluators import ClusteringEvaluator
from mteb.evaluation import MTEB
from mteb.abstasks import AbsTaskRetrieval

data = df.Description

In [ ]:
# def preprocess_text(text):
#     text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
#     text = ' '.join(text.split())
#     doc = nlp(text)
#     sentences = [i.text.strip() for i in doc.sents if len(i) > 1]
#     return sentences

# # Define custom model class
# class MyModel():
#     def encode(self, data, **kwargs):
#         tf.compat.v1.reset_default_graph()
#         tf.compat.v1.disable_eager_execution()

#         url = "https://tfhub.dev/google/elmo/3"
#         embed = hub.Module(url)

#         sentences = []
#         for text in data:
#             preprocessed_sentences = preprocess_text(text)
#             sentences.extend(preprocessed_sentences)

#         embeddings = embed(
#             sentences,
#             signature="default",
#             as_dict=True)["default"]

#         with tf.compat.v1.Session() as sess:
#             sess.run(tf.compat.v1.global_variables_initializer())
#             sess.run(tf.compat.v1.tables_initializer())
#             x = sess.run(embeddings)

#         return x

# # Instantiate your model
# model = MyModel()

# # Instantiate the MTEB framework
# evaluation = MTEB(tasks=["TwentyNewsgroupsClustering"])

# # Run the evaluation
# evaluation.run(model, output_folder="results/elmo")

In [ ]:
def preprocess_text(text):
    text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
    text = ' '.join(text.split())
    doc = nlp(text)
    sentences = [i.text.strip() for i in doc.sents if len(i) > 1]
    return sentences

class MyModel():
    def encode(self, data, **kwargs):
        url = "https://tfhub.dev/google/elmo/3"
        embed = hub.load(url)

        sentences = []
        for text in data:
            preprocessed_sentences = preprocess_text(text)
            sentences.extend(preprocessed_sentences)

        with tf.Graph().as_default():
            text_input = tf.placeholder(dtype=tf.string, shape=[None])
            embeddings = embed(text_input)
            
            with tf.compat.v1.Session() as sess:
                sess.run(tf.compat.v1.global_variables_initializer())
                sess.run(tf.compat.v1.tables_initializer())
                x = sess.run(embeddings, feed_dict={text_input: sentences})
        
        return x

# Instantiate your model
model = MyModel()

# Instantiate the MTEB framework
evaluation = MTEB(tasks=["HotpotQA"])

# Run the evaluation
evaluation.run(model, output_folder="results/elmo")

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- HotpotQA, beir, s2p

  0%|          | 0/5233329 [00:00<?, ?it/s]

In [6]:
def preprocess_text(text):
    text = text.lower().replace('\n', ' ').replace('\t', ' ').replace('\xa0',' ')
    text = ' '.join(text.split())
    doc = nlp(text)
    sentences = [i.text.strip() for i in doc.sents if len(i) > 1]
    return sentences

# Define custom model class
class MyModel():
    def encode(self, data, **kwargs):
        tf.compat.v1.reset_default_graph()
        tf.compat.v1.disable_eager_execution()

        url = "https://tfhub.dev/google/elmo/3"
        embed = hub.Module(url)

        sentences = []
        for text in data:
            preprocessed_sentences = preprocess_text(text)
            sentences.extend(preprocessed_sentences)

        # Reduce batch size and disable GPU usage
        with tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(device_count={'GPU': 0})) as sess:
            sess.run(tf.compat.v1.global_variables_initializer())
            sess.run(tf.compat.v1.tables_initializer())
            x = sess.run(embed)

        return x
    
model = MyModel()

# Instantiate the MTEB framework
evaluation = MTEB(tasks=["ArguAna"])
evaluation.run(model, output_folder=f"results/elmo")

───────────────────────────────────────────────── Selected tasks  ─────────────────────────────────────────────────

Retrieval

- ArguAna, beir, p2p

/root/.cache/huggingface/datasets/BeIR/arguana.zip:   0%|          | 0.00/3.60M [00:00<?, ?iB/s]

  0%|          | 0/8674 [00:00<?, ?it/s]

TypeError: Argument `fetch` = <tensorflow_hub.module.Module object at 0x7c9daabace80> has invalid type "Module" must be a string or Tensor. (Can not convert a Module into a Tensor or Operation.)